# Importing FB Prophet for Time Series Forecasting

In [1]:
# check prophet version
import fbprophet
# print version number
print('Prophet %s' % fbprophet.__version__)

Importing plotly failed. Interactive plots will not work.


Prophet 0.7.1


Loading the Data

In [ ]:
# load the car sales dataset
from pandas import read_csv
# load data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0)
# summarize shape
print(df.shape)
# show first few rows
print(df.head())

Visualizing the dataset

In [ ]:
# plot the car sales dataset
from matplotlib import pyplot
# plot the time series
df.plot()
pyplot.show()

# Fitting the FB Prophet model

In [ ]:
# fit prophet model on the car sales dataset
from pandas import to_datetime
from fbprophet import Prophet

# prepare expected column names
df.columns = ['ds', 'y']
df['ds']= to_datetime(df['ds'])
# define the model
model = Prophet()
# fit the model
model.fit(df)

In [ ]:
from pandas import DataFrame
# define the period for which we want a prediction
future = list()
for i in range(1, 13):
	date = '1968-%02d' % i
	future.append([date])
future = DataFrame(future)
future.columns = ['ds']
future['ds']= to_datetime(future['ds'])